对于BSM微分方程:
$$
\frac{\partial f}{\partial t} + bS\frac{\partial f}{\partial S} + \frac{1}{2}\sigma^2S^2\frac{\partial^2 f}{\partial S^2} = rf \tag{5.1}
$$

香草期权可以获得其解析解，但是大部分期权很难通过解微分方程得到答案，因此，蒙特卡洛成为了价格模拟的一种方法，而除此之外，我们也可以使用有限差分方法来近似解。

有限差分法定价即通过有限差分的方法，解期权价值相关的偏微分方程，得到期权价值的数值解。这与微积分的切割思想相通，对于希腊字母而言：

$delta = \frac{\partial f}{\partial S} = \frac{f_{S+\Delta S}-f_{S-\Delta S}}{2\Delta S}$

$gamma = \frac{\partial^2 f}{\partial S^2} = \frac{f_{S+\Delta S}-2f_{S}+f_{S-\Delta S}}{\Delta S^2}$

$theta = \frac{\partial f}{\partial t} = \frac{f_{t+\Delta t}-f_{t-\Delta t}}{2\Delta t}$

上式解为有限差分法的体现。


构建t-S网格，i表示第i个时间点，j表示第j个价格点,即$S(i,j)=j\Delta S,t(i,j)=i\Delta t$，在空间维度上，即对S的差分基本采用中间差分法,而时间维度上，采取后向差分:

$$delta = \frac{f_{i,j+1}-f_{i,j-1}}{2\Delta S}$$

$$gamma = \frac{f_{i,j+1}-2f_{i,j}+f_{i,j-1}}{\Delta S^2}$$

$$
theta = \frac{f_{i,j}-f_{i-1,j}}{\Delta t}
$$

带入(5.1)，得:
$$
\frac{f_{i,j}-f_{i-1,j}}{\Delta t} + bj\Delta S\frac{f_{i,j+1}-f_{i,j-1}}{2\Delta S} + \frac{1}{2}\sigma^2j^2\Delta S^2\frac{f_{i,j+1}-2f_{i,j}+f_{i,j-1}}{\Delta S^2} = rf_{i-1,j}\\
\tag{5.2}
$$

整理得:
$$
f_{i-1,j} = a_jf_{i,j-1}+b_jf_{i,j}+c_jf_{i,j+1}\\
其中:\\
a_j = \frac{-0.5bj\Delta t + 0.5\sigma^2j^2\Delta t}{1+r\Delta t}\\
b_j = \frac{1-\sigma^2j^2\Delta t}{1+r\Delta t}\\
c_j = \frac{0.5bj\Delta t + 0.5\sigma^2j^2\Delta t}{1+r\Delta t} \tag{5.3}
$$


In [ ]:
#网格法
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import pandas as pd
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #正常显示负号
@jit
def explicit_FD(CP,S,K,T,sigma,r,b,M,N):
    """
    显式有限差分法，是否收敛取决于dt，ds，相对来说不太稳定
               f[i+1,j+1]
    f[i,j] ⬅  f[i,j+1]
               f[i-1,j+1]

    Parameters
    ----------
    CP : "C","P"表示看涨或看跌
    S : 标的价格
    K : 行权价
    sigma:波动率
    r:无风险利率
    b:持有成本，当b = r 时，为标准的无股利模型，b=0时，为black76，b为r-q时，为支付股利模型，b为r-rf时为外汇期权.持有成本
    M : 初始给定的价格节点数量
    N: 初始给定的时间节点数量
    Returns
    期权价值
    """

    ds = S/M   # 确定价格步长，分子用S的意义在于可以让S必定落在网格点上，后续不需要使用插值法
    M = int(K/ds) *4  #确定覆盖的价格范围，这里设置为4倍的行权价，也可根据需要设置为其他，这里根据价格范围重新计算价格点位数量M
    S_idx= int(S / ds)   # S所在的index，用于方便确定初始S对应的期权价值
    print(S_idx)
    dt = T/N  # 时间步长
    df = 1 / (1 + r*dt) #折现因子
    print(f"生产的网格：价格分为M = {M}个点位，时间分为N = {N}个点位")
    V_grid = np.zeros((M+1,N+1)) # 预先生成包括0在内的期权价值矩阵

    S_array = np.linspace(0, M*ds,M+1)  #价格序列
    T_array = np.linspace(0, N*dt,N+1)  #时间序列
    T2M_array = T_array[-1] -T_array    #生成到期时间的数组，方便后面计算边界条件
    if CP == "C":
        V_grid[:,N] = np.maximum(S_array - K,0)  #确定终值条件，到期时期权价值很好计算
        V_grid[M] = np.exp(-r*T2M_array) * (S_array[-1] * np.exp(b*T2M_array) - K)   # 上边界价格够高，期权表现像远期，这里是远期定价，而不是简单得S-X
    else:
        V_grid[:,N] = np.maximum(K - S_array ,0)  #确定终值条件，到期时期权价值很好计算
        V_grid[0] = np.exp(-r*T2M_array) * K


    for i in range(N-1,-1,-1): #时间倒推循环
        for j in range(1,M):  # 价格循环
            # 确定系数
            aj = 0.5 * (sigma**2 * j**2 - b*j) * dt
            bj = 1 - sigma**2 * j**2 * dt
            cj = 0.5 * (sigma**2 * j**2 + b*j) * dt
            # 显示也可以用矩阵来写，但是直接循环也比较直观
            V_grid[j,i] = df * (aj * V_grid[j-1,i+1] + bj * V_grid[j,i+1] + cj * V_grid[j+1,i+1])
            # 美式期权提前行权判断，欧式期权可以直接删除下面的代码
            if CP == "C":
                V_grid[j,i] = max(S_array[j] - K, V_grid[j,i])  #美式期权提前行权判断
            else:
                V_grid[j,i] = max(K - S_array[j], V_grid[j,i])      
    return V_grid[S_idx,0] # 返回初0时点的初始价格的价值

In [6]:
explicit_FD(CP = "P",S = 36,K = 40,T = 0.5,sigma = 0.4,r = 0.06,b =0.06,M = 125,N=50000)

125
生产的网格：价格分为M = 552个点位，时间分为N = 50000个点位


5.9667421297107754

以上采用网格法计算，本质上来说，这种方法是计算一种三叉树，但不考虑每个分支的概率， 而是把每个时间点的每个价格点的价值都计算出来，最后找到第0期下当前S所在的价值点，取该价值点的价格作为期权价值。

现在考虑将网格法转换为矩阵法，在构建方法时，我们构建了一个M+1行N+1列的矩阵V_grid，用来存储每个时间点每个价格点的期权价值。而由于0价值点和M价值点的价值我们采用的是边界条件，在循环开始前就已经确定，因此，真正需要计算的是M-1个数值，有

\begin{equation}
\left[
\begin{array}{c}
V_{i,1}\\V_{i,2}\\V_{i,3}\\ \vdots \\V_{i,M-2}\\V_{i,M-1}
\end{array}
\right]
=
\left[
\begin{array}{ccccc}
b_1 & c_1 & 0 & \cdots & 0 \\
a_2 & b_2 & c_2 & \cdots & 0 \\
0 & a_3 & b_3 & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
0 & \vdots  & a_{M-2} & b_{M-2} & c_{M-2} \\
0 & 0 & 0 & a_{M-1} & b_{M-1}
\end{array}
\right]
\left[
\begin{array}{c}
V_{i+1,1}\\V_{i+1,2}\\V_{i+1,3}\\ \vdots \\V_{i+1,M-2}\\V_{i+1,M-1}
\end{array}
\right]
+
\left[
\begin{array}{c}
a_1V_{i+1,0}\\0\\0\\ \vdots \\0\\c_{M-1}V_{i+1,M}
\end{array}
\right]

\end{equation}

In [7]:
#如何制作对角矩阵？
np.diag(np.ones(5))

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [10]:
np.diag(np.ones(4),-1)

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [11]:
np.diag(np.ones(4),-2)

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [12]:
np.diag(np.ones(4),1)

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])

In [13]:
np.diag(np.ones(4),1)+np.diag(np.ones(4),-1)+np.diag(np.ones(5))

array([[1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.]])

In [20]:
def gen_diag(M,a,v,c,Sd_idx=0):#生成M-1维度的d对角矩阵，a为对角线左下方的值，b为对角的值，c为对角线右上方的值
    a_m_1 = [a(i) for i in range(Sd_idx+2,M)]
    b_m_1 = [v(i) for i in range(Sd_idx+1,M)]
    c_m_1 = [c(i) for i in range(Sd_idx+1,M-1)]
    return np.diag(a_m_1,-1)+np.diag(b_m_1,0)+np.diag(c_m_1,1)


In [15]:
sigma = 0.4
b = 0.06
dt = 0.01
aj = lambda i: 0.5*(sigma**2*i**2-b*i)*dt
bj = lambda i: 1-sigma**2*i**2*dt
cj = lambda i: 0.5*(sigma**2*i**2+b*i)*dt

In [21]:
print(np.round(gen_diag(7,aj,bj,cj),4))

[[0.9984 0.0011 0.     0.     0.     0.    ]
 [0.0026 0.9936 0.0038 0.     0.     0.    ]
 [0.     0.0063 0.9856 0.0081 0.     0.    ]
 [0.     0.     0.0116 0.9744 0.014  0.    ]
 [0.     0.     0.     0.0185 0.96   0.0215]
 [0.     0.     0.     0.     0.027  0.9424]]


In [24]:
#以下生成矩阵法下的有限差分美式期权定价
def explicit_FD_M(CP,S,K,T,sigma,r,b,M,N):
    ds = S/M
    M = int(K/ds)*4
    S_idx = int(S/ds)
    dt = T/N
    df = 1/(1+r*dt)
    print(f"生产的网格:价格分为M={M}个,时间分为N={N}个点位")
    V_grid = np.zeros((M+1,N+1))

    S_array = np.linspace(0,M*ds,M+1)
    T_array = np.linspace(0,N*dt,N+1)
    T2M_array = T_array[-1] - T_array

    if CP=='C':
        V_grid[:,N] = np.maximum(S_array-K,0)
        V_grid[M] = np.exp(-b*T2M_array)*(S_array[-1]*np.exp(-b*T2M_array)-K)
    else:
        V_grid[:,N] = np.maximum(K-S_array,0)
        V_grid[M] = np.exp(-b*T2M_array)*K
    
    aj = lambda i: 0.5*(sigma**2*i**2-b*i)*dt
    bj = lambda i: 1-sigma**2*i**2*dt
    cj = lambda i: 0.5*(sigma**2*i**2+b*i)*dt

    coef_matrix  = gen_diag(M,aj,bj,cj)

    for j in range(N-1,-1,-1):
        Z = np.zeros_like(V_grid[1:M,j+1])
        Z[0] = aj(1)*V_grid[0,j+1]
        Z[-1] = cj(M-1)*V_grid[-1,j+1]
        #矩阵求解
        V_grid[1:M,j] = df*(coef_matrix@V_grid[1:M,j+1] + Z)
        #美式期权提前行权判定
        if CP=='C':
            V_grid[1:M,j] = np.maximum(V_grid[1:M,j],S_array[1:M]-K)
        else:
            V_grid[1:M,j] = np.maximum(V_grid[1:M,j],K-S_array[1:M])
    return V_grid[S_idx,0]

In [25]:
explicit_FD_M(CP = "P",S = 36,K = 40,T = 0.5,sigma = 0.4,r = 0.06,b =0.06,M = 125,N=50000)

生产的网格:价格分为M=552个,时间分为N=50000个点位


5.966747025924541

In [27]:
#不管是普通网格点，还是矩阵法，都是显式差分法，其对于S和T划分要求较高，若划分的不够，会出现发散的情况
#例如
explicit_FD_M(CP = "P",S = 36,K = 40,T = 0.5,sigma = 0.4,r = 0.06,b =0.06,M = 125,N=10000)

生产的网格:价格分为M=552个,时间分为N=10000个点位


nan

由于显示差分法的发散问题，现在介绍隐式和半隐式差分法，这类方法收敛更快：

隐式中对于时间差分采用前向差分法,即：

$$
theta = \frac{f_{i+1,j}-f_{i,j}}{\Delta t}
$$

带入BSM微分方程：
$$
f_{i+1,j} = a_jf_{i,j-1} + b_jf_{i,j}+c_jf_{i,j+1}\\
其中:
a_j = 0.5bj\Delta t - 0.5\sigma^2j^2\Delta t\\
b_j = 1 + (r+\sigma^2j^2)\Delta t\\
c_j = -0.5bj\Delta t - 0.5\sigma^2j^2\Delta t\\
$$

考虑矩阵形式，对于1到M-1的网格点，有:

\begin{equation}
\left[
\begin{array}{c}
V_{i+1,1}\\V_{i+1,2}\\V_{i+1,3}\\ \vdots \\V_{i+1,M-2}\\V_{i+1,M-1}
\end{array}
\right]
=
\left[
\begin{array}{ccccc}
b_1 & c_1 & 0 & \cdots & 0 \\
a_2 & b_2 & c_2 & \cdots & 0 \\
0 & a_3 & b_3 & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
0 & \vdots  & a_{M-2} & b_{M-2} & c_{M-2} \\
0 & 0 & 0 & a_{M-1} & b_{M-1}
\end{array}
\right]
\left[
\begin{array}{c}
V_{i,1}\\V_{i,2}\\V_{i,3}\\ \vdots \\V_{i,M-2}\\V_{i,M-1}
\end{array}
\right]
+
\left[
\begin{array}{c}
a_1V_{i,0}\\0\\0\\ \vdots \\0\\c_{M-1}V_{i,M}
\end{array}
\right]

\end{equation}

上面矩阵简写为:

$$
V_{i+1} = MV_{i} + Z_i
$$

因而有:
$$
V_i = M^{-1}(V_{i+1} - Z_i)
$$

In [ ]:
def implicit_FD(CP,S,K,T,sigma,r,b,M,N):
    """
    隐式有限差分法，比显示更容易收敛，因此N直接指定也容易收敛,但需要通过求逆矩阵解方程组
    """
    ds = S/M   
    M = int(K/ds) *2  
    S_idx= int(S / ds)   
    dt = T/N  
    T_array = np.linspace(0, N*dt,N+1)  #时间序列
    T2M_array = T_array[-1] -T_array
    print(f"生产的网格：价格分为M = {M}个点位，时间分为N = {N}个点位")

    V_grid = np.zeros((M+1,N+1)) 
    S_array = np.linspace(0, M*ds,M+1)  
    if CP == "C":
        V_grid[:,N] = np.maximum(S_array - K,0)  
        V_grid[M] = np.exp(-r*T2M_array) * (S_array[-1] * np.exp(b*T2M_array) - K)
    else:
        V_grid[:,N] = np.maximum(K - S_array ,0)  
        V_grid[0] = np.exp(-r*T2M_array) * K       


    #定义方程的系数的算法，方便后面计算，而且也比较直观
    aj = lambda i : 0.5 * i * (b - sigma**2 * i) * dt
    bj = lambda i : 1 + (r + sigma**2 * i**2) * dt
    cj = lambda i : 0.5 * i * (-b - sigma**2 * i) * dt

    #用自定义的函数gen_diag有效减少代码
    coefficient_matrix = gen_diag(M, aj, bj, cj)
    M_inverse = np.linalg.inv(coefficient_matrix)     

    for j in range(N-1,-1,-1): #隐式也是时间倒推循环，区别在于隐式是要解方程组
        # 准备好解方程组 fj = M**-1 * fj+1,M就是coefficient_matrix的逆矩阵，fj+1的第一项和最后一项需要减去pd*V_grid(0,j)和pu*V_grid(M,j)
        Z = np.zeros_like(V_grid[1:M,j])  
        Z[0] = aj(1) * V_grid[0,j]  
        Z[-1] = cj(M-1) * V_grid[-1,j]
        V_grid[1:M,j] = M_inverse @ (V_grid[1:M,j+1]-Z)

        # 美式期权提前行权判断
        if CP == "C":
            V_grid[1:M,j] = np.maximum(S_array[1:M] - K, V_grid[1:M,j])
        else:
            V_grid[1:M,j] = np.maximum(K - S_array[1:M], V_grid[1:M,j])
    return V_grid[S_idx,0] 

In [30]:
implicit_FD(CP = "P",S = 36,K = 40,T = 0.5,sigma = 0.4,r = 0.06,b =0.06,M = 500,N = 2000)
#dt的划分远低于显式，相同的结果精度，更快的运算速度

生产的网格：价格分为M = 1110个点位，时间分为N = 2000个点位


5.966470922077616

半隐式需要通过求平均点来得到中点的差分,在这种差分方式下，希腊字母为：

$$
delta = \frac{1}{2}(\frac{f_{i,j+1}-f_{i,j-1}}{2\Delta S} + \frac{f_{i+1,j+1}-f_{i+1,j-1}}{2\Delta S})\\
gamma = \frac{1}{2}(\frac{f_{i,j+1}-2f_{i,j}+f_{i,j-1}}{\Delta S^2} + \frac{f_{i+1,j+1}-2f_{i+1,j}+f_{i+1,j-1}}{\Delta S^2})\\
theta = \frac{f_{i+1,j}-f_{i,j}}{\Delta t}
$$

依旧是的带入BSM微分方程，得到:
$$
-a_jf_{i,j-1} + (1-b_j)f_{i,j} - c_jf_{i,j+1} = a_j f_{i+1,j-1} + (1+b_j)f_{i+1,j}+c_jf_{i+1,j+1}\\
其中:\\
a_j = 0.25\Delta t(-bj+\sigma^2j^2)\\
b_j = -0.5\Delta t(r+\sigma^2j^2)\\
c_j = 0.25\Delta t(bj+\sigma^2j^2)
$$

依旧是矩阵形式:

\begin{equation}
\left[
\begin{array}{ccccc}
1-b_1 & -c_1 & 0 & \cdots & 0 \\
-a_2 & 1-b_2 & -c_2 & \cdots & 0 \\
0 & -a_3 & 1-b_3 & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
0 & \vdots  & -a_{M-2} & 1-b_{M-2} & -c_{M-2} \\
0 & 0 & 0 & -a_{M-1} & 1-b_{M-1}
\end{array}
\right]

\left[
\begin{array}{c}
V_{i,1}\\V_{i,2}\\V_{i,3}\\ \vdots \\V_{i,M-2}\\V_{i,M-1}
\end{array}
\right]
=
\left[
\begin{array}{ccccc}
1+b_1 & c_1 & 0 & \cdots & 0 \\
a_2 & 1+b_2 & c_2 & \cdots & 0 \\
0 & a_3 & 1+b_3 & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \vdots \\
0 & \vdots  & a_{M-2} & 1+b_{M-2} & c_{M-2} \\
0 & 0 & 0 & a_{M-1} & 1+b_{M-1}
\end{array}
\right]
\left[
\begin{array}{c}
V_{i+1,1}\\V_{i+1,2}\\V_{i+1,3}\\ \vdots \\V_{i+1,M-2}\\V_{i+1,M-1}
\end{array}
\right]
+
\left[
\begin{array}{c}
a_1[V_{i+1,0}+V_{i,0}]\\0\\0\\ \vdots \\0\\c_{M-1}[V_{i+1,M}+V_{i,M}]
\end{array}
\right]

\end{equation}


即:
$$
M_1V_i = M_2V_{i+1} + Z_{i,i+1}\\
V_i = M_1^{-1}(M_2V_{i+1} + Z_{i,i+1})
$$

In [31]:
def CN_FD(CP,S,K,T,sigma,r,b,M,N):
    """
    半隐式有限差分法，Crank_Nicolson，最稳定的方法，推荐
    """
    ds = S/M   
    M = int(K/ds) *2  
    S_idx= int(S / ds)  
    dt = T/N  
    T_array = np.linspace(0, N*dt,N+1)  #时间序列
    T2M_array = T_array[-1] -T_array
    print(f"生产的网格：价格分为M = {M}个点位，时间分为N = {N}个点位")

    V_grid = np.zeros((M+1,N+1)) 
    S_array = np.linspace(0, M*ds,M+1)  
    if CP == "C":
        V_grid[:,N] = np.maximum(S_array - K,0) 
        V_grid[M] = np.exp(-r*T2M_array) * (S_array[-1] * np.exp(b*T2M_array) - K) 
    else:
        V_grid[:,N] = np.maximum(K - S_array ,0)  
        V_grid[0] = np.exp(-r*T2M_array) * K       



    aj = lambda i : 0.25 * (sigma**2 * i**2 - b * i) * dt
    bj = lambda i : -0.5 * (r + sigma**2 * i**2) * dt
    cj = lambda i : 0.25 * (sigma**2 * i**2 + b * i) * dt
    matrix_ones = np.diag([1 for i in range(M-1)])
    matrix_1 = - gen_diag(M, aj, bj, cj) + matrix_ones
    matrix_2 = gen_diag(M,aj, bj, cj) + matrix_ones
    M1_inverse = np.linalg.inv(matrix_1)

    for j in range(N-1,-1,-1): 
        # 准备好解方程组 M_1 * fj = M_2 * fj+1 + b_1
        # Z是对边界条件的处理
        Z = np.zeros_like(V_grid[1:M,j + 1])
        Z[0] = aj(1) * (V_grid[0,j] + V_grid[0,j+1])
        Z[-1] = cj(M-1) * (V_grid[-1,j] + V_grid[-1,j+1]) 

        V_grid[1:M,j] = M1_inverse @ ( matrix_2 @ V_grid[1:M,j + 1]  + Z )
        # 美式期权提前行权判断
        if CP == "C":
            V_grid[1:M,j] = np.maximum(S_array[1:M] - K, V_grid[1:M,j])
        else:
            V_grid[1:M,j] = np.maximum(K - S_array[1:M], V_grid[1:M,j])
    return V_grid[S_idx,0]  


In [32]:
CN_FD(CP = "P",S = 36,K = 40,T = 0.5,sigma = 0.4,r = 0.06,b =0.06,M = 500,N = 2000)

生产的网格：价格分为M = 1110个点位，时间分为N = 2000个点位


5.966945832276144